<a href="https://colab.research.google.com/github/ibonetc/LLM_EPM/blob/main/LLM_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers de Preguntas-Respuestas

In [1]:
!pip install -q -U accelerate peft transformers
!pip install -q bitsandbytes datasets sentencepiece einops gradio trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### google/flan-t5-larg

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [3]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
input_text = "Who is the current UK Prime Minister? Explain step by step"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

In [ ]:
outputs = model.generate(input_ids,max_new_tokens=1000)

In [ ]:
print(tokenizer.decode(outputs[0]))

<pad> The Prime Minister of the United Kingdom is Theresa May. Theresa May is the wife of David Cameron. David Cameron is the Prime Minister of the United Kingdom. So, the final answer is Theresa May.</s>


In [ ]:
input_text = "Who is the president in Colombia? Explain the political situation in the country"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids,max_new_tokens=1000)
print(tokenizer.decode(outputs[0]))

<pad> President of Colombia</s>


## Modelos con template para chat
Algunos modelos están más actualizados que otros. Además permiten un template de chat para el tokenizer.

Acá vemos un ejemplo

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, platform, gradio, warnings
from datasets import load_dataset
from trl import SFTTrainer

### meta-llama/Meta-Llama-3-8B-Instruct


In [6]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert in history!"},
    {"role": "user", "content": "Who is the current UK Prime Minister? Explain step by step"},
]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=1000,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

I'd be happy to explain who the current UK Prime Minister is and how they came to power. Here's a step-by-step guide:

1. The United Kingdom is a parliamentary democracy, which means that the head of government is the Prime Minister, who is responsible for advising the monarch (currently Queen Elizabeth II) on matters of state.

2. The Prime Minister is the leader of the political party or coalition of parties that has the most seats in the House of Commons, which is the lower chamber of the UK Parliament.

3. The House of Commons is made up of 650 elected Members of Parliament (MPs), who are chosen by the public to represent their constituencies.

4. The Prime Minister is typically the leader of the party that wins the most seats in a general election, which is held every five years or whenever the government loses a vote of confidence.

5. In the 2019 general election, the Conservative Party, led by Boris Johnson, won a majority of 80 seats, allowing them to form a government.

6. Bo

In [ ]:
messages_2 = [
    {"role": "system", "content": "You are an spanish professor and always answer in Spanish"},
    {"role": "user", "content": "Who is the current Prime Minister of the UK? Explain step by step"},
]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages_2,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

¡Hola! Como profesor de español, me alegra explicarte quién es el actual Primer Ministro del Reino Unido. ¡Vamos a hacerlo paso a paso!

Paso 1: El Reino Unido es un país ubicado en Europa occidental, compuesto por Inglaterra, Escocia, Gales y Irlanda del Norte.

Paso 2: El sistema político del Reino Unido es una monarquía constitucional, lo que significa que el monarca (actualmente, Isabel II) es la jefa de Estado, pero no tiene poder ejecutivo. En su lugar, el Primer Ministro es el jefe de gobierno.

Paso 3: El Primer Ministro del Reino Unido es elegido por el partido político que tenga la mayoría en el Parlamento del Reino Unido, que es la cámara legislativa del país.

Paso 4: En las elecciones generales del Reino Unido, los ciudadanos eligen a los miembros del Parlamento (conocidos como diputados) que representan a sus respectivos distritos electorales.

Paso 5: El partido político que tenga la mayoría


In [ ]:
input_text = "Who is the current UK Prime Minister? Explain step by step"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids,max_new_tokens=1000,pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

<|begin_of_text|>Who is the UK Prime Minister? Explain step by step
The UK Prime Minister is the head of government in the United Kingdom. Here's a step-by-step guide on how they are chosen:
Step 1: The General Election
The UK holds a general election every five years, unless the government loses a vote of no confidence or the House of Commons is dissolved. During the election, voters elect members of parliament (MPs) to represent their constituencies.

Step 2: The Party with the Most Seats
The party with the most seats in the House of Commons forms the government. This is usually the party with the largest number of MPs.

Step 3: The Leader of the Winning Party
The leader of the party with the most seats becomes the Prime Minister. This is usually the leader of the party that won the most seats in the general election.

Step 4: The Queen's Approval
The Queen must approve the appointment of the Prime Minister. This is a formal process, and the Queen meets with the leader of the winning

In [ ]:
input_text = "You are an spanish professor and always answer in Spanish. Who is the current UK Prime Minister? Explain step by step"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids,max_new_tokens=1000,pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

### HuggingFaceH4/zephyr-7b-bet

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta")

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

As of my training data (last updated in March 2021), the current UK Prime Minister is Boris Johnson. Here's a step-by-step explanation:

1. The UK is a constitutional monarchy, which means that the monarch (currently Queen Elizabeth II) is the head of state, but the day-to-day running of the government is carried out by the Prime Minister, who is the head of the government.

2. The Prime Minister is appointed by the monarch, on the advice of the leader of the political party or coalition that has a majority in the House of Commons (the lower house of the UK Parliament).

3. Boris Johnson is a member of the Conservative Party, which won a large majority in the December 2019 general election.

4. Following the election, Johnson was invited by the Queen to form a new government, and he was sworn in as Prime Minister on 24 July 2019.

5. Johnson's appointment was expected, as he had been the frontrunner to replace Theresa May, who resigned as Prime Minister in May 2019 after failing to get

### mistralai/Mistral-7B-Instruct-v0.3

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert in history!"},
    {"role": "user", "content": "Who is the current UK Prime Minister? Explain step by step"},
]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## Traductor, por defecto de inglés a español.

### facebook/m2m100_418M

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

In [ ]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

In [ ]:
outputs = model.generate(input_ids,max_new_tokens=1000)

In [ ]:
print(tokenizer.decode(outputs[0]))

</s> __es__ ¿Quién es el primer ministro del Reino Unido? explica paso a paso</s>


# Chat con contexto

In [ ]:
from transformers import pipeline

In [ ]:
context = """
La inteligencia artificial (IA) es una rama de la ciencia de la computación que se encarga de crear sistemas
capaces de realizar tareas que normalmente requieren inteligencia,
como el reconocimiento de voz, la toma de decisiones y la traducción de idiomas.
"""

In [ ]:
questions = [
    "¿Qué es la inteligencia artificial?",
    "¿Qué tareas puede realizar la IA?",
    "¿Qué aplicaciones tiene la inteligencia artificial?"
]

In [ ]:
qa_model = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
for question in questions:
  result = qa_model(question=question, context=context)
  print(f"Pregunta: {question}")
  print(f"Respuesta: {result['answer']}\n")

Pregunta: ¿Qué es la inteligencia artificial?
Respuesta:  una rama de la ciencia de la computación

Pregunta: ¿Qué tareas puede realizar la IA?
Respuesta:  la toma de decisiones y la traducción de idiomas.

Pregunta: ¿Qué aplicaciones tiene la inteligencia artificial?
Respuesta:  la toma de decisiones y la traducción de idiomas.



# Fine tuning

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

## Usar modelo

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model="NousResearch/Llama-2-7b-chat-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt = "What is a large language model?"
result = pipe(f"<s>[INST] {prompt} [/INST]",max_new_tokens=1000)

In [ ]:
result

[{'generated_text': '<s>[INST] What is a large language model? [/INST]  A large language model'}]

In [ ]:
print(result[0]['generated_text'])

## Cargar tokenizer y modelo con la configuración QLoRA

### Cargar base de datos

In [ ]:
dataset_name = "mlabonne/guanaco-llama2-1k"

In [ ]:
dataset = load_dataset(dataset_name, split="train")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

### Cargar modelo

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

Parámetros SFT

#### Parámetros bitsandbytes

In [ ]:
use_4bit = True # Activate 4-bit precision base model loading
bnb_4bit_compute_dtype = "float16" # Compute dtype for 4-bit base models
bnb_4bit_quant_type = "nf4" # Quantization type (fp4 or nf4)
use_nested_quant = False # Activate nested quantization for 4-bit base models (double quantization)

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Chequear compatibilidad GPU con bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Modelo base

In [ ]:
max_seq_length = None # Maximum sequence length to use
packing = False # Pack multiple short examples in the same input sequence to increase efficiency
device_map = {"": 0} # Load the entire model on the GPU 0

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Cargar LLama tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
tokenizer.padding_side = 'right'

Parámetros QLoRA

In [ ]:
lora_r = 64 # LoRA attention dimension
lora_alpha = 16 # Alpha parameter for LoRA scaling
lora_dropout = 0.1 # Dropout probability for LoRA layers

Cargar configuración LoRA

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

### Entrenamiento

Parámetros de entrenamiento

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard"
)

Parámetros de fine-tuning

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/l

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Entrenar el modelo

In [ ]:
trainer.train()

Cargar el modelo

In [ ]:
trainer.model.save_pretrained('llama_fine_tuning')